In [39]:
import pandas as pd
import string

x = pd.read_csv("billboard_100.csv")

genius_characters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789- '

def format_for_genius(string):
    formatted_string = ''
    for character in string:
        if character in genius_characters:
            formatted_string += character
        elif character == '&':
            formatted_string += 'and'
    return formatted_string

def remove_features(string):
    for i in range(len(string)):
        if string[i:i+9] == 'featuring':
            string = string[:i-1]
    return string

title = []
artists = []

for i in range(len(x)):
    title.append(format_for_genius(x['Title'][i]))
    artists.append(format_for_genius(remove_features(x['Artist(s)'][i])))

print(title)






['Thrift Shop', 'Blurred Lines', 'Radioactive', 'Harlem Shake', 'Cant Hold Us', 'Mirrors', 'Just Give Me a Reason', 'When I Was Your Man', 'Cruise', 'Roar', 'Locked Out of Heaven', 'Ho Hey', 'Stay', 'Get Lucky', 'Royals', 'I Knew You Were Trouble', 'We Cant Stop', 'Wrecking Ball', 'Wake Me Up', 'Suit and Tie', 'Cups Pitch Perfects When Im Gone', 'Holy Grail', 'Scream and Shout', 'Clarity', 'Sail', 'Dont You Worry Child', 'Diamonds', 'I Love It', 'Safe and Sound', 'Treasure', 'The Way', 'Started from the Bottom', 'Come and Get It', 'Hold On Were Going Home', 'Daylight', 'Feel This Moment', 'Applause', 'One More Night', 'Love Me', 'My Songs Know What You Did in the Dark Light Em Up', 'Fuckin Problems', 'Beauty and a Beat', 'Same Love', 'Sweet Nothing', 'Summertime Sadness', 'Home', 'Its Time', 'Power Trip', 'Girl on Fire', 'Heart Attack', 'Love Somebody', 'I Will Wait', 'Try', 'Wagon Wheel', 'Gangnam Style', 'I Need Your Love', 'Die Young', 'Some Nights', 'Bad', 'Boys Round Here', 'Gone 